In [58]:
import pandas as pd
import seaborn as sns
#pip install openpyxl 

#### Pregunta N ° 1 - Objetivos:

Elaborar una tabla con la facturación de cada producto considerando la zona.  

In [59]:
# Cargamos como un DataFrame (DF) los datos a analizar

df = pd.read_excel("FACT_TOTAL.XLSX", sheet_name = "BASE")
df

# Cargamos el Excel donde iran las respuestas
pregunta1 = pd.read_excel("pregunta1.xlsx")

#### Analisis general


In [60]:
## Declaracion de variables

id_zona = df['CODIGO_ZONA']
id_comprador = df['CODIGO_DE_COMPRADOR']
cantidad_pedida = df['CANTIDAD_PEDIDA']
cantidad_despachada = df['CANTIDAD_DESPACHADA']
id_producto = df['CODIGO_DEL_PRODUCTO']

In [61]:
## Informacion sobre las columnas

df.info() # los ids de los productos son de tipo objeto. Se procede a analizar los datos de la columna.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 169305 entries, 0 to 169304
Data columns (total 5 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   CODIGO_ZONA          169305 non-null  int64 
 1   CODIGO_DE_COMPRADOR  169305 non-null  int64 
 2   CANTIDAD_PEDIDA      169305 non-null  int64 
 3   CANTIDAD_DESPACHADA  169305 non-null  int64 
 4   CODIGO_DEL_PRODUCTO  169305 non-null  object
dtypes: int64(4), object(1)
memory usage: 6.5+ MB


In [62]:
## Analisis estadistico 

df.describe()

,CODIGO_ZONA,CODIGO_DE_COMPRADOR,CANTIDAD_PEDIDA,CANTIDAD_DESPACHADA
count,169305.000000,169305.000000,169305.000000,169305.000000
mean,322.621511,333875.442592,2.999368,2.498284
std,172.744022,188629.636448,0.816227,0.958236
min,101.000000,37.000000,2.000000,1.000000
25%,157.000000,196568.000000,2.000000,2.000000
50%,311.000000,324571.000000,3.000000,2.000000
75%,407.000000,436874.000000,4.000000,3.000000
max,828.000000,999389.000000,4.000000,4.000000


In [ ]:
## Valores de las distintas columnas

print(set(id_producto)) # (!) tiene caracteres entre los numeros
print(set(id_zona)) # ok
print(set(cantidad_pedida)) # ok
print(set(cantidad_despachada)) #ok

## Ver cuantas zonas y cuantos productos hay

print(len(set(id_zona))) #163 Zonas de reparto unicas
print(len(set(id_producto))) #801 Productos unicos

#### Eliminar caracteres y convertir los ids de producto en enteros

In [64]:
id_producto.replace({'X':'','V':''}, regex=True, inplace=True)

In [ ]:
set(id_producto)

In [66]:
df = df.astype({"CODIGO_DEL_PRODUCTO" : int})

In [67]:
df.dtypes

CODIGO_ZONA            int64
CODIGO_DE_COMPRADOR    int64
CANTIDAD_PEDIDA        int64
CANTIDAD_DESPACHADA    int64
CODIGO_DEL_PRODUCTO    int64
dtype: object

#### Preparar los datos

In [68]:
ventas_por_producto_y_zona = pd.DataFrame(df.groupby(['CODIGO_DEL_PRODUCTO', 'CODIGO_ZONA'])['CANTIDAD_PEDIDA'].sum().sort_values(ascending = False))
ventas_por_producto_y_zona

CANTIDAD_PEDIDA
CODIGO_DEL_PRODUCTO CODIGO_ZONA                 
469519              612                     2324
283760              331                     1976
469519              150                     1872
                    272                     1781
283760              343                     1719
...                                          ...
860129              619                        2
751674              606                        2
288159              116                        2
558312              334                        2
317803              513                        2

[48534 rows x 1 columns]

In [69]:
despachos_por_producto_y_zona = pd.DataFrame(df.groupby(['CODIGO_DEL_PRODUCTO', 'CODIGO_ZONA'])['CANTIDAD_DESPACHADA'].sum().sort_values(ascending = False))
despachos_por_producto_y_zona.sort_values('CODIGO_ZONA')

,,CANTIDAD_DESPACHADA
CODIGO_DEL_PRODUCTO,CODIGO_ZONA,
584511,101,5
147213,101,6
829010,101,2
39633,101,6
299808,101,22
...,...,...
861703,828,1
504969,828,3
234196,828,2


In [107]:
compradores_por_zona = pd.DataFrame(df.groupby(['CODIGO_ZONA'])['CODIGO_DE_COMPRADOR'].nunique())
compradores_por_zona

##compradores_por_zona.to_excel(pregunta1,sheet_name= "compradores_por_zona", index = False)

,CODIGO_DE_COMPRADOR
CODIGO_ZONA,
101,472
102,359
103,590
104,452
107,400
...,...
824,8
825,1
826,1


In [108]:
compradores_por_zona.rename({'CODIGO_DE_COMPRADOR': 'NUM_COMPRADORES'},axis=1,inplace=True)
compradores_por_zona

,NUM_COMPRADORES
CODIGO_ZONA,
101,472
102,359
103,590
104,452
107,400
...,...
824,8
825,1
826,1


In [ ]:
compradores_por_zona.to_excel('compradores.xlsx')

In [ ]:
ventas_por_producto_y_zona["CANTIDAD_DESPACHADA"] = despachos_por_producto_y_zona["CANTIDAD_DESPACHADA"]
ventas_por_producto_y_zona.sort_values('CODIGO_DEL_PRODUCTO')
ventas_por_producto_y_zona = df


In [109]:
df["Variacion pedido-despacho"] = (df["CANTIDAD_DESPACHADA"] - df["CANTIDAD_PEDIDA"])/(df["CANTIDAD_PEDIDA"])*100
df

,CODIGO_ZONA,CODIGO_DE_COMPRADOR,CANTIDAD_PEDIDA,CANTIDAD_DESPACHADA,CODIGO_DEL_PRODUCTO,Variacion pedido-despacho
0,807,633115,2,2,283760,0.000000
1,807,633115,3,2,283760,-33.333333
2,810,309373,2,2,283760,0.000000
3,810,309373,3,3,296648,0.000000
4,810,309373,2,2,306259,0.000000
...,...,...,...,...,...,...
169300,619,113156,2,2,283760,0.000000
169301,619,113156,2,2,283760,0.000000
169302,619,113156,2,1,592959,-50.000000
169303,619,113156,4,3,707645,-25.000000
